In [ ]:
import os
import pickle
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy
import numpy as np

In [ ]:
#List and unpickle all files in "results/"
reports = []
for file in os.listdir("results/"):
    if file.endswith(".pkl"):
        with open("results/"+file, "rb") as f:
            reports.append(pickle.load(f))

def get_series(report, key):
    report = deepcopy(report)
    model_name = report.pop("name")
    series = np.zeros(len(report))
    for k, v in report.items():
        idx = int(k.split('_')[0]) - 2
        series[idx] = v.get(key, 0)
    feature_dims = np.arange(len(report)) + 2
    return feature_dims, series, model_name

In [ ]:
fig, ax = plt.subplots()
for report in reports:
    x, y, model_name = get_series(report, "hausdorff-dist")
    _, dy, _ = get_series(report, "optimality-gap_std")
    ax.plot(x, y, label=model_name)
    ax.fill_between(x, y - dy, y + dy, alpha=0.2)
ax.set_xlabel("Feature dimension")
ax.set_ylabel("Optimality gap")
ax.set_xmargin(0)
ax.legend(frameon=False)